# Keyword extraction for Interactive Image Retrieval

## Setting up and trying things:

Let's work with a toy example for an article now.

### Data

In [1]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

In [2]:
doc

"\n         Supervised learning is the machine learning task of \n         learning a function that maps an input to an output based \n         on example input-output pairs.[1] It infers a function \n         from labeled training data consisting of a set of \n         training examples.[2] In supervised learning, each \n         example is a pair consisting of an input object \n         (typically a vector) and a desired output value (also \n         called the supervisory signal). A supervised learning \n         algorithm analyzes the training data and produces an \n         inferred function, which can be used for mapping new \n         examples. An optimal scenario will allow for the algorithm \n         to correctly determine the class labels for unseen \n         instances. This requires the learning algorithm to  \n         generalize from the training data to unseen situations \n         in a 'reasonable' way (see inductive bias).\n      "

### Candidate Keywords/Keyphrases

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

In [4]:
candidates

array(['algorithm', 'allow', 'analyzes', 'based', 'bias', 'called',
       'class', 'consisting', 'correctly', 'data', 'desired', 'determine',
       'example', 'examples', 'function', 'generalize', 'inductive',
       'inferred', 'infers', 'input', 'instances', 'labeled', 'labels',
       'learning', 'machine', 'mapping', 'maps', 'new', 'object',
       'optimal', 'output', 'pair', 'pairs', 'produces', 'reasonable',
       'requires', 'scenario', 'set', 'signal', 'situations',
       'supervised', 'supervisory', 'task', 'training', 'typically',
       'unseen', 'used', 'value', 'vector', 'way'], dtype=object)

We can use n_gram_range to change the size of the resulting candidates. For example, if we would set it to (3, 3) then the resulting candidates would phrases that include 3 keywords.

We can play around with n_gram_range to create different lengths of keyphrases. Then, we might not want to remove stop_words as they can tie longer keyphrases together.

### Embeddings

Next, we convert both the document as well as the candidate keywords/keyphrases to numerical data. We use BERT for this purpose as it has shown great results for both similarity- and paraphrasing tasks.

In [5]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

We will use Distilbert as it has shown great performance in similarity tasks, which is what we are aiming for with keyword/keyphrase extraction! \
Since transformer models have a token limit, we might run into some errors when inputting large documents. In that case, we cann consider splitting up the document into paragraphs and mean pooling (taking the average of) the resulting vectors.
**NOTE:** There are many pre-trained BERT-based models that we can use for keyword extraction. For our use case the best are either `distilbert—base-nli-stsb-mean-tokens` or `xlm-r-distilroberta-base-paraphase-v1` as they have shown great performance in semantic similarity and paraphrase identification respectively.

### Similarity:

As a final step we want to find the candidates that are most similar to the document. We assume that the most similar candidates to the document are good keywords/keyphrases for representing the document.\
To calculate the similarity between candidates and the document, we will be using the cosine similarity between vectors as it performs quite well in high-dimensionality:

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

top_n = 3
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['training', 'algorithm', 'learning']

The results look great! These terms definitely look like they describe a document about supervised machine learning.
Now, let us take a look at what happens if we change the n_gram_range to (3,3):

In [7]:
n_gram_range = (3, 3)
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()
candidate_embeddings = model.encode(candidates)
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['learning machine learning',
 'learning algorithm analyzes',
 'algorithm generalize training']

We notice that the keywords are too similar, we will need to find a way to diversify this.

### Diversification

We notice that the results are very similar and this is expected as they best represent the document! If we were to diversify the keywords/keyphrases then they are less likely to represent the document well as a collective.
Thus, the diversification of our results requires a delicate balance between the accuracy of keywords/keyphrases and the diversity between them.
There are two algorithms that we will be using to diversify our results:
- Max Sum Similarity
- Maximal Marginal Relevance


#### Max Sum Similarity 
The maximum sum distance between pairs of data is defined as the pairs of data for which the distance between them is maximized. In our case, we want to maximize the candidate similarity to the document whilst minimizing the similarity between candidates.

In [8]:
import numpy as np
import itertools

def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]


To do this, we select some top keywords/keyphrases, and from those, select the 5 that are the least similar to each other

In [9]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['requires learning algorithm',
 'signal supervised learning',
 'learning function maps',
 'algorithm analyzes training',
 'learning machine learning']

-> If we set a **low** `nr_candidates`, then our results seem to be very similar to our original cosine similarity method.

In [10]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=20)

['set training examples',
 'generalize training data',
 'requires learning algorithm',
 'supervised learning algorithm',
 'learning machine learning']

-> However, a relatively **high** `nr_candidates` will create more diverse keyphrases.

#### Maximal Marginal Relevance
We can diversify our results using Maximal Marginal Relevance (MMR). MMR tries to minimize redundancy and maximize the diversity of results in text summarization tasks. \
We start by selecting the keyword/keyphrase that is the most similar to the document. Then, we iteratively select new candidates that are both similar to the document and not similar to the already selected keywords/keyphrases:

In [11]:
def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [12]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['algorithm generalize training',
 'supervised learning algorithm',
 'learning machine learning',
 'learning algorithm analyzes',
 'learning algorithm generalize']

If we set a relatively **low diversity**, then our results seem to be very similar to our original cosine similarity method.

In [13]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['algorithm generalize training',
 'labels unseen instances',
 'new examples optimal',
 'determine class labels',
 'supervised learning algorithm']

However, a relatively **high diversity** score will create very diverse keyphrases.